In [3]:
!pip install --upgrade azure-cognitiveservices-vision-computervision
!pip install pillow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.4/85.4 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 198.9/198.9 kB 3.7 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 151.7/151.7 kB 5.0 MB/s eta 0:00:00


In [4]:
from azure.cognitiveservices.vision.computervision import ComputerVisionClient
from azure.cognitiveservices.vision.computervision.models import OperationStatusCodes
from azure.cognitiveservices.vision.computervision.models import VisualFeatureTypes
from msrest.authentication import CognitiveServicesCredentials

from array import array
import os
from PIL import Image
import sys
import time     

import json

In [6]:
with open('secret.json') as f:
    secret = json.load(f)

In [7]:
KEY = secret['KEY']
ENDPOINT = secret['ENDPOINT']

In [9]:
computervision_client = ComputerVisionClient(ENDPOINT, CognitiveServicesCredentials(KEY))

In [11]:
remote_image_url = "https://raw.githubusercontent.com/Azure-Samples/cognitive-services-sample-data-files/master/ComputerVision/Images/landmark.jpg"
print("===== Describe an image - remote =====")
description_results = computervision_client.describe_image(remote_image_url)

print("Description of remote image: ")
if (len(description_results.captions) == 0):
    print("No description detected.")
else:
    for caption in description_results.captions:
        print("'{}' with confidence {:.2f}%".format(caption.text, caption.confidence * 100))

===== Describe an image - remote =====
Description of remote image: 
'an ancient city with many ruins with Colosseum in the background' with confidence 33.80%


In [12]:
print("===== Category an image - remote =====")

remote_image_features = ["categories"]
category_results = computervision_client.analyze_image(remote_image_url, visual_features=[VisualFeatureTypes.categories])

print("Categories from remote image: ")
if (len(category_results.categories) == 0):
    print("No categories detected.")
else:
    for category in category_results.categories:
        print("'{}' with confidence {:.2f}%".format(category.name, category.score * 100))
        

===== Category an image - remote =====
Categories from remote image: 
'building_' with confidence 31.64%
'others_' with confidence 0.39%
'outdoor_' with confidence 3.91%


In [13]:
print("===== Tags an image - remote =====")
tags_result_remote = computervision_client.tag_image(remote_image_url)  

print("Tags in the remote image: ")
if (len(tags_result_remote.tags) == 0):
    print("No tags detected.")
else:
    for tag in tags_result_remote.tags:
        print("'{}' with confidence {:.2f}%".format(tag.name, tag.confidence * 100))    

===== Tags an image - remote =====
Tags in the remote image: 
'outdoor' with confidence 99.00%
'building' with confidence 98.81%
'sky' with confidence 98.21%
'stadium' with confidence 98.17%
'ancient rome' with confidence 96.16%
'ruins' with confidence 95.04%
'amphitheatre' with confidence 93.99%
'ancient roman architecture' with confidence 92.65%
'historic site' with confidence 89.55%
'ancient history' with confidence 89.54%
'history' with confidence 86.72%
'archaeological site' with confidence 84.41%
'travel' with confidence 65.85%
'large' with confidence 61.02%
'city' with confidence 56.57%


In [15]:
print("===== Detect Objects - remote =====")
remote_image_url_objects = "https://raw.githubusercontent.com/Azure-Samples/cognitive-services-sample-data-files/master/ComputerVision/Images/objects.jpg"
detect_objects_results_remote = computervision_client.detect_objects(remote_image_url_objects)

print("Detecting objects in remote image:")

if len(detect_objects_results_remote.objects) == 0:
    print("No objects detected.")
else:
    for object in detect_objects_results_remote.objects:
        print("object at location {}, {}, {}, {}".format( \
        object.rectangle.x, object.rectangle.x + object.rectangle.w, \
        object.rectangle.y, object.rectangle.y + object.rectangle.h))  


===== Detect Objects - remote =====
Detecting objects in remote image:
object at location 213, 365, 85, 208
object at location 218, 402, 179, 384
object at location 238, 417, 298, 416
object at location 116, 419, 60, 386


In [25]:
local_image_path = "sample01.jpg"
local_image = open(local_image_path, "rb")

print("===== Describe an image - local =====")
description_results_local = computervision_client.describe_image_in_stream(local_image)

print("Description in local image: ")
if (len(description_results_local.captions) == 0):
    print("No description detected.")
else:
    for caption in description_results_local.captions:
        print("'{}' with confidence {:.2f}%".format(caption.text, caption.confidence * 100))

local_image_path = "sample02.jpg"
local_image = open(local_image_path, "rb")

print("===== Describe an image - local =====")
description_results_local = computervision_client.describe_image_in_stream(local_image)

print("Description in local image: ")
if (len(description_results_local.captions) == 0):
    print("No description detected.")
else:
    for caption in description_results_local.captions:
        print("'{}' with confidence {:.2f}%".format(caption.text, caption.confidence * 100))


===== Describe an image - local =====
Description in local image: 
'a fountain in a pond' with confidence 41.05%
===== Describe an image - local =====
Description in local image: 
'a city with bright lights at night' with confidence 41.79%


In [27]:
def get_tags(filepath):
    local_image = open(filepath, "rb")
    tags_result = computervision_client.tag_image_in_stream(local_image)
    tags = tags_result.tags
    tags_name = [tag.name for tag in tags]

    return tags_name

local_image_path = "sample01.jpg"
get_tags(local_image_path)

['outdoor',
 'sky',
 'tree',
 'fountain',
 'water',
 'cloud',
 'plant',
 'lake',
 'grass',
 'park']

In [29]:
def detect_objects(filepath):        
    local_image = open(filepath, "rb")
    detect_objects_results = computervision_client.detect_objects_in_stream(local_image)
    objects = detect_objects_results.objects

    return objects

local_image_path = "sample01.jpg"
objects = detect_objects(local_image_path)
objects[0].rectangle.x

50